##### Copyright 2019 The TensorFlow Authors.

In [6]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [8]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [9]:
Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

SyntaxError: invalid syntax (<ipython-input-9-0ec2870e922b>, line 1)

In [10]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [11]:
predictions = model(x_train[:1]).numpy()
predictions


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[-0.12587005,  0.3222917 ,  0.4083975 , -0.69591606,  0.5779902 ,
         0.33620748, -0.2947482 ,  0.14865571, -0.05761461,  0.37845013]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [12]:
tf.nn.softmax(predictions).numpy()

array([[0.07500157, 0.11740985, 0.12796754, 0.04241334, 0.15161876,
        0.11905512, 0.06334721, 0.09869505, 0.08029959, 0.12419205]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [13]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [14]:
loss_fn(y_train[:1], predictions).numpy()

2.1281688

In [15]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [16]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 2s 31us/sample - loss: 0.2939 - accuracy: 0.9153
Epoch 2/5
60000/60000 [==============================] - 2s 26us/sample - loss: 0.1432 - accuracy: 0.9571
Epoch 3/5
60000/60000 [==============================] - 2s 26us/sample - loss: 0.1078 - accuracy: 0.9667
Epoch 4/5
60000/60000 [==============================] - 2s 25us/sample - loss: 0.0901 - accuracy: 0.9722
Epoch 5/5
60000/60000 [==============================] - 1s 24us/sample - loss: 0.0778 - accuracy: 0.9758


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)".

In [17]:
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 0s - loss: 0.0757 - accuracy: 0.9766


[0.07571280423700809, 0.9766]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [18]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [19]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[3.3112311e-08, 1.0679660e-10, 1.5298347e-06, 4.9852908e-05,
        1.3415481e-12, 5.6433478e-08, 2.0410066e-15, 9.9993074e-01,
        7.2887883e-07, 1.7043369e-05],
       [8.4958414e-09, 4.4948210e-05, 9.9995387e-01, 7.0604693e-07,
        4.7206200e-16, 2.0429539e-08, 3.3798185e-07, 5.6376462e-12,
        1.3809728e-07, 1.4120967e-18],
       [2.5274989e-07, 9.9749315e-01, 3.0134237e-04, 8.1377721e-06,
        3.0718224e-05, 4.5567015e-07, 4.1962776e-05, 1.8549706e-03,
        2.6756129e-04, 1.5989481e-06],
       [9.9985850e-01, 3.5976021e-11, 1.0608456e-04, 1.6484546e-06,
        2.2842837e-09, 4.7630496e-07, 1.6054404e-05, 1.4465627e-05,
        3.2361643e-07, 2.4338151e-06],
       [8.6862865e-06, 2.6423022e-10, 1.7026975e-04, 9.9083502e-07,
        9.8466426e-01, 7.4568663e-07, 6.1010551e-06, 4.8240108e-04,
        6.3326265e-06, 1.4660174e-02]], dtype=float32)>